# CourtSense MVP - Tennis Recovery Analyzer

This notebook runs the tennis player tracking and coaching analysis on Google Colab.

## Step 1: Setup Environment

In [ ]:
# Install dependencies
!pip install -q sam2 opencv-python-headless matplotlib numpy torch

# Clone the repo
!git clone https://github.com/fish5421/tennis_vision_app.git
%cd tennis_vision_app

## Step 2: Upload Your Video

In [ ]:
from google.colab import files
import shutil

print("Upload your tennis video file:")
uploaded = files.upload()

# Move to data folder
video_filename = list(uploaded.keys())[0]
shutil.move(video_filename, f"data/{video_filename}")
VIDEO_PATH = f"data/{video_filename}"
print(f"\nVideo saved to: {VIDEO_PATH}")

## Step 3: Extract First Frame & Display

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

# Read first frame
cap = cv2.VideoCapture(VIDEO_PATH)
ret, frame = cap.read()
cap.release()

if not ret:
    raise ValueError(f"Could not read video: {VIDEO_PATH}")

# Convert BGR to RGB for matplotlib
frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
frame_height, frame_width = frame.shape[:2]

print(f"Video dimensions: {frame_width} x {frame_height}")

# Display the frame
plt.figure(figsize=(14, 10))
plt.imshow(frame_rgb)
plt.title("First frame - note the court corners for next step")
plt.axis('on')
plt.show()

## Step 4: Interactive Court Calibration

**Click the 4 corners of the singles court in this order:**
1. Bottom-Left (closest to camera, left side)
2. Bottom-Right (closest to camera, right side)  
3. Top-Right (far end, right side)
4. Top-Left (far end, left side)

**Wait for 4 clicks, then close the plot or wait for timeout.**

In [ ]:
%matplotlib notebook
# Note: If %matplotlib notebook doesn't work, try %matplotlib inline and use the manual entry cell below

import matplotlib.pyplot as plt
import numpy as np

print("="*60)
print("CLICK 4 COURT CORNERS in order:")
print("  1. Bottom-Left (near camera, left)")
print("  2. Bottom-Right (near camera, right)")
print("  3. Top-Right (far end, right)")
print("  4. Top-Left (far end, left)")
print("="*60)

fig, ax = plt.subplots(figsize=(14, 10))
ax.imshow(frame_rgb)
ax.set_title("Click 4 court corners (BL -> BR -> TR -> TL)")

# Collect 4 points interactively
court_corners = plt.ginput(4, timeout=120)  # 2 minute timeout
plt.close()

if len(court_corners) == 4:
    court_corners = np.array(court_corners, dtype=np.float32)
    print(f"\n✅ Court corners captured:")
    print(f"   Bottom-Left:  ({court_corners[0][0]:.0f}, {court_corners[0][1]:.0f})")
    print(f"   Bottom-Right: ({court_corners[1][0]:.0f}, {court_corners[1][1]:.0f})")
    print(f"   Top-Right:    ({court_corners[2][0]:.0f}, {court_corners[2][1]:.0f})")
    print(f"   Top-Left:     ({court_corners[3][0]:.0f}, {court_corners[3][1]:.0f})")
else:
    print(f"❌ Only got {len(court_corners)} points. Need 4. Run this cell again.")

### Alternative: Manual Corner Entry
If interactive clicking doesn't work, enter coordinates manually:

In [ ]:
# MANUAL ENTRY - Uncomment and edit if interactive mode doesn't work
# Look at the frame image above and estimate pixel coordinates

# court_corners = np.array([
#     [100, 650],   # Bottom-Left (x, y)
#     [1100, 650],  # Bottom-Right
#     [900, 300],   # Top-Right
#     [300, 300],   # Top-Left
# ], dtype=np.float32)

print(f"Current court_corners: {court_corners}")

## Step 5: Interactive Player Bounding Box Selection

**Click 2 points to define a box around the player:**
1. Top-left corner of box
2. Bottom-right corner of box

In [ ]:
%matplotlib notebook

print("="*60)
print("CLICK 2 POINTS to draw a box around the player:")
print("  1. Top-left corner of bounding box")
print("  2. Bottom-right corner of bounding box")
print("="*60)

fig, ax = plt.subplots(figsize=(14, 10))
ax.imshow(frame_rgb)
ax.set_title("Click top-left then bottom-right of player bounding box")

bbox_points = plt.ginput(2, timeout=120)
plt.close()

if len(bbox_points) == 2:
    x1, y1 = bbox_points[0]
    x2, y2 = bbox_points[1]
    bbox = (int(min(x1,x2)), int(min(y1,y2)), int(abs(x2-x1)), int(abs(y2-y1)))
    print(f"\n✅ Bounding box: x={bbox[0]}, y={bbox[1]}, w={bbox[2]}, h={bbox[3]}")
else:
    print("❌ Need 2 points. Using auto-bbox instead.")
    bbox = None

### Alternative: Manual BBox Entry

In [ ]:
# MANUAL ENTRY - Uncomment and edit if interactive mode doesn't work

# bbox = (500, 400, 150, 300)  # (x, y, width, height)

print(f"Current bbox: {bbox}")

## Step 6: Verify Selections Visually

In [ ]:
%matplotlib inline

# Draw court corners and bbox on frame
vis = frame_rgb.copy()

# Draw court polygon
pts = court_corners.astype(np.int32)
for i, pt in enumerate(pts):
    color = [(255,0,0), (0,255,0), (0,0,255), (255,255,0)][i]
    cv2.circle(vis, tuple(pt), 10, color, -1)
    cv2.putText(vis, f"{i+1}", (pt[0]+15, pt[1]), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
cv2.polylines(vis, [pts], True, (0, 255, 0), 2)

# Draw bbox
if bbox:
    x, y, w, h = bbox
    cv2.rectangle(vis, (x, y), (x+w, y+h), (255, 0, 255), 3)
    cv2.putText(vis, "Player", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2)

plt.figure(figsize=(14, 10))
plt.imshow(vis)
plt.title("Verify: Green=Court, Magenta=Player BBox")
plt.show()

print("\nDoes this look correct? If not, re-run Steps 4 or 5.")

## Step 7: Compute Homography

In [ ]:
import cv2
import numpy as np

# Tennis court dimensions in meters
COURT_WIDTH = 8.23
COURT_LENGTH = 23.77

# Destination points (real-world court corners)
# Order: Bottom-Left, Bottom-Right, Top-Right, Top-Left
dst_points = np.array([
    [0.0, 0.0],
    [COURT_WIDTH, 0.0],
    [COURT_WIDTH, COURT_LENGTH],
    [0.0, COURT_LENGTH]
], dtype=np.float32)

# Compute homography
H, _ = cv2.findHomography(court_corners, dst_points)

print("✅ Homography matrix computed:")
print(H)

# Save for later use
np.save("data/homography.npy", H)
np.save("data/court_corners.npy", court_corners)
print("\nSaved to data/homography.npy and data/court_corners.npy")

## Step 8: Run Tracking & Analysis

In [ ]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
from modules.tracker import PlayerTracker
from modules.calibration import pixel_to_meter
from modules import geometry, coach
import numpy as np

# Load homography
H = np.load("data/homography.npy")

# Initialize tracker
print("Loading SAM2 model...")
tracker = PlayerTracker(model_id="facebook/sam2-hiera-small")

# Run tracking
print(f"\nTracking player in {VIDEO_PATH}...")
bbox_str = f"{bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]}" if bbox else None

tracking_frames, fps = tracker.track(
    video_path=VIDEO_PATH,
    display=False,  # No display in Colab
    store_masks=False,
    homography=H,
    bbox=bbox,
    allow_bbox_fallback=True,
    max_frames=300,  # Adjust based on video length
)

print(f"\n✅ Tracked {len(tracking_frames)} frames at {fps:.1f} FPS")

## Step 9: Convert to Meter Coordinates & Analyze

In [ ]:
# Convert pixel coords to meters
def fill_coords(tracking_frames, h_matrix):
    coords_m = []
    last_valid = None
    for tf in tracking_frames:
        if tf.centroid_px is not None:
            metr = pixel_to_meter([tf.centroid_px], h_matrix)[0]
            last_valid = metr
        if last_valid is None:
            coords_m.append(np.array([0.0, 0.0]))
        else:
            coords_m.append(last_valid)
    return coords_m

coords_m = fill_coords(tracking_frames, H)
coords_m = np.array(coords_m)

# Validate
valid_centroids = sum(1 for tf in tracking_frames if tf.centroid_px is not None)
print(f"Valid centroids: {valid_centroids}/{len(tracking_frames)}")
print(f"X range: {coords_m[:,0].min():.2f} to {coords_m[:,0].max():.2f} m")
print(f"Y range: {coords_m[:,1].min():.2f} to {coords_m[:,1].max():.2f} m")

# Save
np.save("data/coords_m.npy", coords_m)
print("\nSaved to data/coords_m.npy")

In [ ]:
# Detect shots and compute recovery
shot_frames = geometry.detect_shots(coords_m, fps)
print(f"Detected {len(shot_frames)} shots")

ideal_pos = geometry.compute_dynamic_ideal_position(coords_m)
print(f"Dynamic ideal position: ({ideal_pos[0]:.2f}m, {ideal_pos[1]:.2f}m)")

recovery_stats = geometry.compute_recovery_times(coords_m, shot_frames, fps, ideal_position=ideal_pos)
print(f"\nRecovery stats for {len(recovery_stats)} shots:")
for stat in recovery_stats:
    print(f"  Shot {stat['shot_id']}: {stat['time_to_recover']:.2f}s")

## Step 10: Generate Coaching Feedback

In [ ]:
# Generate coaching feedback (heuristic - no Ollama in Colab)
feedback = coach.generate_feedback(recovery_stats, use_ollama=False, coords_m=coords_m)

print("="*60)
print("COACHING FEEDBACK")
print("="*60)
print(feedback)

## Step 11: Visualize Top-Down Path

In [ ]:
%matplotlib inline

# Render top-down view
topdown = geometry.render_topdown_path(coords_m, shot_frames)

if topdown is not None:
    # Convert BGR to RGB
    topdown_rgb = cv2.cvtColor(topdown, cv2.COLOR_BGR2RGB)
    
    plt.figure(figsize=(8, 12))
    plt.imshow(topdown_rgb)
    plt.title("Player Movement Path (Top-Down View)")
    plt.axis('off')
    plt.show()
    
    # Save
    cv2.imwrite("data/topdown_path.png", topdown)
    print("Saved to data/topdown_path.png")

## Step 12: Download Results

In [ ]:
from google.colab import files

# Download the results
files.download("data/topdown_path.png")
files.download("data/coords_m.npy")